In [41]:
import pandas as pd
import numpy as np

# Workflow Description
blabla

In [3]:
# Load the data files
file_path_female = 'data/remaining-life-expectancy-at-different-ages-females.csv'
file_path_male = 'data/remaining-life-expectancy-at-different-ages-males.csv'

female_data = pd.read_csv(file_path_female)
male_data = pd.read_csv(file_path_male)

#### Create basic df structure with clear target variable of remaining-life-expectancy-at-different-ages-females and males

In [11]:
# Extracting age columns and reshaping the female data
female_reshaped = female_data.melt(id_vars=['Entity', 'Year', 'Code'],
                                   var_name='Age', value_name='Remaining Life Expectancy')
female_reshaped['Sex'] = 'Female'

# Extracting age columns and reshaping the male data, including 'Code'
male_reshaped = male_data.melt(id_vars=['Entity', 'Year', 'Code'],
                               var_name='Age', value_name='Remaining Life Expectancy')
male_reshaped['Sex'] = 'Male'

# Concatenate the reshaped dataframes with 'Code'
combined_data = pd.concat([female_reshaped, male_reshaped])

# Extract the age from the 'Age' column
combined_data['Age'] = combined_data['Age'].str.extract(r'Age: (\d+)').astype(int)

# Reordering columns for clarity
combined_data = combined_data[['Entity', 'Code', 'Year', 'Sex', 'Age', 'Remaining Life Expectancy']]

combined_data.head()

,Entity,Code,Year,Sex,Age,Remaining Life Expectancy
0,Afghanistan,AFG,1950,Female,0,28.3905
1,Afghanistan,AFG,1951,Female,0,28.6345
2,Afghanistan,AFG,1952,Female,0,29.1260
3,Afghanistan,AFG,1953,Female,0,29.6212
4,Afghanistan,AFG,1954,Female,0,29.9004


Based on an analysis of the available datasets, we have decided to limit the data range to the years 2000 to 2020. This will help us manage the amount of missing data (NaN) more effectively.

In [12]:
# Filtering the data for years between 2000 and 2020 and removing rows without a 'Code' variable beacuse theyre not Countries
filtered_data = combined_data.dropna(subset=['Code'])
filtered_data = filtered_data[(filtered_data['Year'] >= 2000) & (filtered_data['Year'] <= 2020)]

filtered_data.head()



,Entity,Code,Year,Sex,Age,Remaining Life Expectancy
50,Afghanistan,AFG,2000,Female,0,56.8579
51,Afghanistan,AFG,2001,Female,0,57.3437
52,Afghanistan,AFG,2002,Female,0,57.6715
53,Afghanistan,AFG,2003,Female,0,58.6498
54,Afghanistan,AFG,2004,Female,0,59.2159


#### Adding more data

In [32]:
# Load the additional data files
file_path_education_expenditure = 'data/total-government-expenditure-on-education-gdp.csv'
file_path_internet_usage = 'data/share-of-individuals-using-the-internet.csv'
file_path_electricity_access = 'data/share-of-the-population-with-access-to-electricity.csv'
file_path_sanitation = 'data/share-using-safely-managed-sanitation.csv'
file_path_smoking = 'data/share-of-adults-who-smoke.csv'
file_path_gdp = 'data/gdp.csv'
file_path_meat_supply = 'data/meat-supply-per-person.csv'
file_path_obesity = 'data/obesity.csv'
file_path_healthcare_spending = 'data/public-healthcare-spending-share-gdp.csv'
file_path_air_pollution = 'data/pm25-air-pollution.csv'
file_path_democracy_index = 'data/DemocracyIndex.csv'

# Reading the data files into DataFrames
education_expenditure_data = pd.read_csv(file_path_education_expenditure)
internet_usage_data = pd.read_csv(file_path_internet_usage)
electricity_access_data = pd.read_csv(file_path_electricity_access)
sanitation_data = pd.read_csv(file_path_sanitation)
smoking_data = pd.read_csv(file_path_smoking)
gdp_data = pd.read_csv(file_path_gdp)
meat_supply_data = pd.read_csv(file_path_meat_supply)
obesity_data = pd.read_csv(file_path_obesity)
healthcare_spending_data = pd.read_csv(file_path_healthcare_spending)
air_pollution_data = pd.read_csv(file_path_air_pollution)

# Reading and selecting specific columns from the Democracy Index file
democracy_index_data = pd.read_csv(file_path_democracy_index)[['Entity', 'Code', 'Year', 'Electoral democracy index']]

# Joining all these dataframes with the filtered_data dataframe based on 'Entity' and 'Year'
# Only join when the year exists in the filtered_data
# Adding custom suffixes to handle duplicate column names
merged_data = filtered_data
for df in [education_expenditure_data, internet_usage_data, electricity_access_data, 
           sanitation_data, smoking_data, gdp_data, meat_supply_data, 
           obesity_data, healthcare_spending_data, air_pollution_data, democracy_index_data]:
    merged_data = merged_data.merge(df, on=['Entity', 'Year'], how='left', suffixes=('', '_extra'))

# Dropping extra columns created due to overlapping column names (if any)
columns_to_drop = [col for col in merged_data.columns if '_extra' in col]
merged_data.drop(columns=columns_to_drop, inplace=True)

merged_data.head(10)



,Entity,Code,Year,Sex,Age,Remaining Life Expectancy,Gov expenditure on education (%),Internet usage (% of population),Access to electricity (% of population),SanitationAccess,SmokingRateAdults,GDP ($),Meat consumptionm in kg per year per capita,ObesityRate (BMI > 30),Healthcare spending (% of GDP),"air pollution, annual exposure (micrograms per cubic meter)",Electoral democracy index
0,Afghanistan,AFG,2000,Female,0,56.8579,NaN,NaN,1.613591,NaN,36.9,NaN,16.344915,1.9,NaN,64.92246,0.072
1,Afghanistan,AFG,2001,Female,0,57.3437,NaN,0.004723,4.074574,NaN,NaN,NaN,14.084803,2.0,NaN,NaN,0.082
2,Afghanistan,AFG,2002,Female,0,57.6715,NaN,0.004561,9.409158,NaN,NaN,2.689005e+10,14.495209,2.1,0.084181,NaN,0.223
3,Afghanistan,AFG,2003,Female,0,58.6498,NaN,0.087891,14.738506,NaN,NaN,2.926506e+10,13.168231,2.2,0.650963,NaN,0.230
4,Afghanistan,AFG,2004,Female,0,59.2159,NaN,0.105809,20.064968,NaN,NaN,2.967890e+10,13.960530,2.3,0.542926,NaN,0.240
5,Afghanistan,AFG,2005,Female,0,59.6966,NaN,1.224148,25.390894,NaN,32.7,3.301176e+10,14.268150,2.5,0.529184,64.52268,0.335
6,Afghanistan,AFG,2006,Female,0,60.2431,4.684761,2.107124,30.718690,NaN,NaN,3.478033e+10,11.292855,2.6,0.497840,NaN,0.397
7,Afghanistan,AFG,2007,Female,0,60.9090,4.174895,1.900000,36.051010,NaN,NaN,3.958917e+10,10.894277,2.8,0.559948,NaN,0.397
8,Afghanistan,AFG,2008,Female,0,61.5289,4.383672,1.840000,42.400000,NaN,NaN,4.114304e+10,12.428057,2.9,0.627800,NaN,0.403
9,Afghanistan,AFG,2009,Female,0,61.8922,4.810640,3.550000,46.740050,NaN,NaN,4.994375e+10,11.999750,3.1,0.531903,NaN,0.392


## Descriptive Statistics

In [39]:
print("Basic Information:")
print(merged_data.info())


Basic Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69678 entries, 0 to 69677
Data columns (total 17 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Entity                                                       69678 non-null  object 
 1   Code                                                         69678 non-null  object 
 2   Year                                                         69678 non-null  int64  
 3   Sex                                                          69678 non-null  object 
 4   Age                                                          69678 non-null  int32  
 5   Remaining Life Expectancy                                    69678 non-null  float64
 6   Gov expenditure on education (%)                             42168 non-null  float64
 7   Internet usage (% of population)                         

In [36]:
print("\nDescriptive Statistics for Numerical Columns:")
print(merged_data.describe())


Descriptive Statistics for Numerical Columns:
               Year           Age  Remaining Life Expectancy  \
count  69678.000000  69678.000000               69678.000000   
mean    2010.000000     34.285714                  42.456526   
std        6.055344     27.701227                  23.711364   
min     2000.000000      0.000000                   3.186500   
25%     2005.000000     10.000000                  17.869150   
50%     2010.000000     25.000000                  48.142650   
75%     2015.000000     65.000000                  62.488700   
max     2020.000000     80.000000                  88.727200   

       Gov expenditure on education (%)  Internet usage (% of population)  \
count                      42168.000000                      57274.000000   
mean                           4.461809                         33.971653   
std                            1.874060                         30.020213   
min                            0.127174                          0.0

In [38]:
print("\nMissing Values in Each Column:")
missing_values = merged_data.isnull().sum()
print(missing_values)


Missing Values in Each Column:
Entity                                                             0
Code                                                               0
Year                                                               0
Sex                                                                0
Age                                                                0
Remaining Life Expectancy                                          0
Gov expenditure on education (%)                               27510
Internet usage (% of population)                               12404
Access to electricity (% of population)                         7280
SanitationAccess                                               29232
SmokingRateAdults                                              53606
GDP ($)                                                        13328
Meat consumptionm in kg per year per capita                    17682
ObesityRate (BMI > 30)                                         23982
He

In [43]:
print("\nPercentage of Missing Values in Each Column:")
missing_values_percent = (merged_data.isnull().sum() / 69678) * 100
print(missing_values_percent)



Percentage of Missing Values in Each Column:
Entity                                                          0.000000
Code                                                            0.000000
Year                                                            0.000000
Sex                                                             0.000000
Age                                                             0.000000
Remaining Life Expectancy                                       0.000000
Gov expenditure on education (%)                               39.481615
Internet usage (% of population)                               17.801889
Access to electricity (% of population)                        10.448061
SanitationAccess                                               41.952984
SmokingRateAdults                                              76.933896
GDP ($)                                                        19.127989
Meat consumptionm in kg per year per capita                    25.376733
Obesi

In [42]:
# Filter for numeric columns before calculating the correlation matrix
numeric_data = merged_data.select_dtypes(include=[np.number])
correlation_matrix = numeric_data.corr()

print("\nCorrelation Matrix:")
print(correlation_matrix)


Correlation Matrix:
                                                            Year  \
Year                                                1.000000e+00   
Age                                                -1.238518e-13   
Remaining Life Expectancy                           3.911469e-02   
Gov expenditure on education (%)                    3.596767e-03   
Internet usage (% of population)                    5.454042e-01   
Access to electricity (% of population)             1.172784e-01   
SanitationAccess                                    9.020032e-02   
SmokingRateAdults                                  -3.072737e-01   
GDP ($)                                             2.551573e-02   
Meat consumptionm in kg per year per capita         4.985795e-02   
ObesityRate (BMI > 30)                              1.900977e-01   
Healthcare spending (% of GDP)                      8.256974e-02   
air pollution, annual exposure (micrograms per ... -3.229558e-02   
Electoral democracy index  

## Data cleansing (NaN handling)